In [75]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import learning_curve
import keras
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf

# Data Processing

In [76]:
filepath = '../../Datasets/SICK/SICK.txt'
data = pd.read_csv(filepath, sep='\t')

In [77]:
data.head()

,pair_ID,sentence_A,sentence_B,entailment_label,relatedness_score,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset,SemEval_set
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,NEUTRAL,4.5,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","A group of children playing in a yard, a man i...",FLICKR,FLICKR,TRAIN
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,NEUTRAL,3.2,A_contradicts_B,B_neutral_A,"A group of children playing in a yard, a man i...","A group of children playing in a yard, a man i...",FLICKR,FLICKR,TRAIN
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,ENTAILMENT,4.7,A_entails_B,B_entails_A,"The children are playing outdoors, while a man...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRAIN
3,4,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,CONTRADICTION,3.6,A_contradicts_B,B_contradicts_A,"The children are playing outdoors, while a man...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRIAL
4,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,NEUTRAL,3.4,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRAIN


In [78]:
num_sentences = data.shape[0]
num_labels = 3

We will extract the Premise and Hypothesis along with the Entailment Label to construct our dataset.

In [79]:
X = data[['sentence_A','sentence_B']]
Y = data['entailment_label']

We can process our label encoding now, but our sentences are not in a format which can be encoded.

In [80]:
le = preprocessing.LabelEncoder().fit(Y)
Y = np_utils.to_categorical(le.transform(Y))
print(Y)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


sentence_A holds the Premise while sentence_B holds the Hypothesis data.

To tokenize our corpus we will need to stack these two columns into a single series.

In [81]:
stacked = pd.concat([X['sentence_A'], X['sentence_B']], axis=0)

In [82]:
# Convert question corpus into sequential encoding for LSTM
vocab_size = 512
sequence_length = 16
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(stacked)
sequences = tokenizer.texts_to_sequences(stacked)
x_text = sequence.pad_sequences(sequences, maxlen=sequence_length)

premise_sequences = x_text[:data.shape[0]]
hypothesis_sequences = x_text[data.shape[0]:]

# Neural Network

In [83]:
epochs = 100
batch_size = 64
learning_rate = .001
dense_units = 128
dropout_rate = .5

In [84]:
embedding_layer = Embedding(vocab_size, 128)
premise_inputs = Input(shape=(sequence_length,))
premise_embedding = embedding_layer(premise_inputs)
premise_outputs, state_h, state_c = LSTM(128, return_state=True)(premise_embedding)

hypothesis_inputs = Input(shape=(sequence_length,))
hypothesis_embedding = embedding_layer(hypothesis_inputs)
hypothesis_outputs = LSTM(128)(hypothesis_embedding, initial_state=[state_h, state_c])

dense = Dense(dense_units, activation='relu', kernel_initializer=keras.initializers.he_normal(seed=None))(hypothesis_outputs)
bn = BatchNormalization()(dense)
dropout = Dropout(dropout_rate)(bn)
outputs = Dense(num_labels, activation='softmax')(dropout)

In [85]:
model = Model(inputs=[premise_inputs, hypothesis_inputs], outputs=outputs)
print(model.summary())

model.compile(optimizer=Adam(lr=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit([premise_sequences, hypothesis_sequences], Y, batch_size=batch_size, epochs=epochs, shuffle=True, validation_split = 0.2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 16)           0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 16)           0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 16, 128)      65536       input_15[0][0]                   
                                                                 input_16[0][0]                   
__________________________________________________________________________________________________
lstm_15 (LSTM)                  [(None, 128), (None, 131584      embedding_8[0][0]                
__________

7872/7872 [==============================] - 8s 1ms/step - loss: 0.2138 - acc: 0.9223 - val_loss: 1.5079 - val_acc: 0.6214
Epoch 43/100
7872/7872 [==============================] - 9s 1ms/step - loss: 0.2006 - acc: 0.9215 - val_loss: 1.8448 - val_acc: 0.5955
Epoch 44/100
7872/7872 [==============================] - 12s 2ms/step - loss: 0.1985 - acc: 0.9243 - val_loss: 1.8295 - val_acc: 0.5940
Epoch 45/100
7872/7872 [==============================] - 9s 1ms/step - loss: 0.1830 - acc: 0.9320 - val_loss: 1.8927 - val_acc: 0.6001
Epoch 46/100
7872/7872 [==============================] - 10s 1ms/step - loss: 0.1839 - acc: 0.9299 - val_loss: 1.6528 - val_acc: 0.6057
Epoch 47/100
7872/7872 [==============================] - 8s 1ms/step - loss: 0.1871 - acc: 0.9313 - val_loss: 1.7215 - val_acc: 0.6159
Epoch 48/100
7872/7872 [==============================] - 8s 1ms/step - loss: 0.1725 - acc: 0.9352 - val_loss: 1.7678 - val_acc: 0.6189
Epoch 49/100
7872/7872 [==============================] - 8